In [29]:
from itertools import islice
import matplotlib.pyplot as plt

from tqdm import tqdm
import torch
from torchvision import models, transforms, datasets

In [2]:
inception_transforms = transforms.Compose([
            transforms.Resize(299),
            #transforms.CenterCrop(constants.INPUT_SIZE),
            transforms.ToTensor(),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

In [3]:
unlabeled_celeba = datasets.ImageFolder('imgs_by_label/celeba_unlabeled/', inception_transforms)
unlabeled_celeba

Dataset ImageFolder
    Number of datapoints: 734
    Root location: imgs_by_label/celeba_unlabeled/

In [4]:
unlabeled_celeba_loader = torch.utils.data.DataLoader(
        unlabeled_celeba, batch_size=1, shuffle=True, num_workers=1)

In [30]:
def get_inception_features(img_iter):
    inception_net = models.inception_v3(pretrained=True, transform_input=True)
    
    layers_to_grab = [inception_net.Conv2d_1a_3x3, inception_net.Conv2d_2b_3x3,
                 inception_net.Conv2d_3b_1x1, inception_net.Mixed_5d, inception_net.Mixed_6e,
                 inception_net.Mixed_7c, inception_net.fc]
    
    layer_features = [None for i in range(len(layers_to_grab))]
    
    
    def hook_fn(self, inp, out, container, layer_index):
        #print(layer_index, inp[0].shape, out.shape)

        num_channels = out.shape[1]
        if len(out.shape) > 2:
            #Warning: this will break for batch sizes > 1
            cur_features = out.squeeze().permute(1,2,0).reshape(-1, num_channels)
        else:
            cur_features = out

        if container[layer_index] is None:
            container[layer_index] = cur_features
        else:
            container[layer_index] = torch.cat((container[layer_index], cur_features))

    def hook_fn_i(container, i):
        return lambda self, inp, out: hook_fn(self, inp, out, container, i)

    for i, layer in enumerate(layers_to_grab):
        layer.register_forward_hook(hook_fn_i(layer_features, i))
        
    inception_net.eval()

    for x,y in tqdm(img_iter):
        print(x.shape, y)
        #plt.imshow((x).squeeze().permute(1, 2, 0))
        #plt.show()
        out = inception_net(x)
        #print(out.sum())
        
    return layer_features

In [ ]:
unlabeled_celeba_features = get_inception_features(unlabeled_celeba_loader)

  0%|          | 0/734 [00:00<?, ?it/s]

torch.Size([1, 3, 299, 299]) tensor([0])


  0%|          | 1/734 [00:00<06:33,  1.86it/s]

torch.Size([1, 3, 299, 299]) tensor([0])


  0%|          | 2/734 [00:01<06:33,  1.86it/s]

...

torch.Size([1, 3, 299, 299]) tensor([0])


 13%|█▎        | 99/734 [02:07<21:42,  2.05s/it]

torch.Size([1, 3, 299, 299]) tensor([0])


 14%|█▎        | 100/734 [02:10<22:03,  2.09s/it]

torch.Size([1, 3, 299, 299]) tensor([0])


In [24]:
[lf.shape for lf in unlabeled_celeba_features]

[torch.Size([66603, 32]),
 torch.Size([64827, 64]),
 torch.Size([15987, 80]),
 torch.Size([3675, 288]),
 torch.Size([867, 768]),
 torch.Size([192, 2048]),
 torch.Size([3, 1000])]